# guide_bot example
This file is an example of how to use guide_bot to create a large project with several guides and a multidimensional scan.

### Import of guide_bot

In [ ]:
import guide_bot as gb

### Set up of figure of merit
First we set up the figure or merit, here described as a Sample object. We want to investigate both a 1 and 3 cm tall sample area, so we provide a list of sample heights.

In [ ]:
target = gb.Target(width=0.02, height=[0.01, 0.03],
                   div_horizontal=0.75, div_vertical=1.2,
                   min_wavelength=1.5, max_wavelength=3.0,
                   instrument_length=60, target_guide_distance=0.5)

### Set up neutron source for optimization
We set up a moderator for the optimization, here we wish to perform optimizations for a range of moderator heights which we provide as a list.

We also set up a second moderator for which we want to test the optimized guides on.

In [ ]:
moderator = gb.Moderator(name="fom_moderator", width=0.1,
                      height=[0.02, 0.04, 0.06, 0.08, 0.1], guide_start=2.0)

large_moderator = gb.Moderator(name="large_moderator", width=0.2, height=0.2, guide_start=2.0)

### Make a guide_bot project
Using our sample and source descriptions, we set up a guide_bot project. This includes settings for numerical optimizations performed in this run.

In [ ]:
settings = {"maxiter": 400, "swarmsize": 50, "minstep": 2E-4,
            "ncount": 5E6, "ncount_analysis": 1E8, "logfile": True}

first_project = gb.Project(name="first_project", target=target, moderator=moderator,
                       settings=settings, analysis_moderators=large_moderator)

### Describing the first guide
A *guide* object is retrieved from the *project* object, then we build the desired guide geometry by adding guide elements. Here we add a feeder, a gap at 6.5 m from the source, followed by an elliptic guide. The gap has start dimensions of 3 by 4 cm, and a length of 10 cm.

In [ ]:
guide = first_project.new_guide(name="Elliptic_with_feeder")
guide += gb.Elliptic(name="Feeder", m=4)
guide += gb.Gap(name="Chopper_gap", start_point=6.5, length=0.1, start_width=0.03, start_height=0.04)
guide += gb.Elliptic(name="Main_guide", m=3)

### Second guide with ballistic main guide
The second guide we wish to investigate incorperates a straight part in the main guide where we wish to have a constant cross section. In practice we ensure this by defining a parameter describing the width/height and assigning these to the start and end of the straight section.

The main guide after the chopper gap will have an ellitpic defocusing and focusing section, each of which is limited to a length of between 2 and 10 m.

In [ ]:
main_guide_width = gb.RelativeFreeInstrumentParameter("guide_width", 0.005, 0.12)
main_guide_height = gb.RelativeFreeInstrumentParameter("guide_height", 0.005, 0.12)

guide = first_project.new_guide(name="Ballistic_with_feeder")
guide += gb.Elliptic(name="Feeder", m=4)
guide += gb.Gap(name="Chopper_gap", start_point=6.5, length=0.1,
                start_width=0.03, start_height=0.04)
guide += gb.Elliptic(name="defocusing", m=3, length=[2, 10],
                     minor_axis_x=main_guide_width, minor_axis_y=main_guide_height)
guide += gb.Straight(name="straight_section", m=1.5,
                     start_width=main_guide_width, start_height=main_guide_height,
                     end_width=main_guide_width, end_height=main_guide_height)
guide += gb.Elliptic(name="focusing", m=3, length=[2, 10],
                     minor_axis_x=main_guide_width, minor_axis_y=main_guide_height)

### Alternative versions without feeder
We also want to investigate if these beam requirements can be fulfilled without a feeder, so we add two more guide objects describing the same guides only without the feeder.

In [ ]:
guide = first_project.new_guide(name="Elliptic_without_feeder")
guide += gb.Gap(name="Chopper_gap", start_point=6.5, length=0.1,
                start_width=0.03, start_height=0.04)
guide += gb.Elliptic(name="Main_guide", m=3)

In [ ]:
main_guide_width = gb.RelativeFreeInstrumentParameter("guide_width", 0.005, 0.12)
main_guide_height = gb.RelativeFreeInstrumentParameter("guide_height", 0.005, 0.12)

guide = first_project.new_guide(name="Ballistic_without_feeder")
guide += gb.Gap(name="Chopper_gap", start_point=6.5, length=0.1,
                start_width=0.03, start_height=0.04)
guide += gb.Elliptic(name="defocusing", m=3, length=[2, 10],
                     minor_axis_x=main_guide_width, minor_axis_y=main_guide_height)
guide += gb.Straight(name="straight_section", m=1.5,
                     end_width=main_guide_width, end_height=main_guide_height)
guide += gb.Elliptic(name="focusing", m=3, length=[2, 10],
                     minor_axis_x=main_guide_width, minor_axis_y=main_guide_height)

### Preparing the project
Lets get an overview of our project, especially to see the number of optimizations that will be performed.

In [ ]:
print(first_project)

### Writing the project to disk
Since the project contains 40 guide optimizations, we wish to use a cluster. Here we load the SLURM module and provide a configuration file for the ESS DMSC cluster. 

In [ ]:
from guide_bot.cluster import SLURM
DMSC = SLURM.ClusterSLURM(cluster_name="DMSC")

first_project.write(cluster=DMSC)

### Run on local computer
While the project is supposed to be executed on a cluster, it is possible to run the simulations locally. In the next cell the syntax is shown, but commented.

In [ ]:
# At the end we start the optimization of the elliptic guide using the runner
#import os
#cpath = os.getcwd()
#os.chdir(os.path.join(cpath, "first_project", "Ballistic_with_feeder"))
#gb.RunFromFile("Ballistic_with_feeder_sam_height_0_mod_height_4.plk")